In [23]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from skimage import data

model = VGG16(weights='imagenet', include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [28]:
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True


In [38]:

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from skimage import data

from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Sequential, Model
from keras import optimizers

input_tensor = Input(shape=(224, 224, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC層を構築
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(4))

# VGG16とFCを接続
model = Model(input=vgg16.input, output=top_model(vgg16.output))

# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:15]:
    layer.trainable = False

# Fine-tuningのときはSGDの方がよい
model.compile(loss='mse',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  app.launch_new_instance()


In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [22]:
model = VGG16(weights='imagenet', include_top=True)
for i in range(len(model.layers)):
    print(i, model.layers[i])

model.summary()

(0, <keras.engine.topology.InputLayer object at 0x7fed10862610>)
(1, <keras.layers.convolutional.Conv2D object at 0x7fed108bdf50>)
(2, <keras.layers.convolutional.Conv2D object at 0x7fed10850fd0>)
(3, <keras.layers.pooling.MaxPooling2D object at 0x7fed10adfd90>)
(4, <keras.layers.convolutional.Conv2D object at 0x7fed108114d0>)
(5, <keras.layers.convolutional.Conv2D object at 0x7fed10811a50>)
(6, <keras.layers.pooling.MaxPooling2D object at 0x7fed107bf5d0>)
(7, <keras.layers.convolutional.Conv2D object at 0x7fed107e2c90>)
(8, <keras.layers.convolutional.Conv2D object at 0x7fed107e22d0>)
(9, <keras.layers.convolutional.Conv2D object at 0x7fed1078a350>)
(10, <keras.layers.pooling.MaxPooling2D object at 0x7fed10799f90>)
(11, <keras.layers.convolutional.Conv2D object at 0x7fed10740c90>)
(12, <keras.layers.convolutional.Conv2D object at 0x7fed10740c10>)
(13, <keras.layers.convolutional.Conv2D object at 0x7fed10768a50>)
(14, <keras.layers.pooling.MaxPooling2D object at 0x7fed10714790>)
(15, <

In [16]:
# img = image.load_img(img_path, target_size=(224, 224))
from skimage.transform import rescale, resize, downscale_local_mean

img = resize(data.coffee(), (224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

decode_predictions(model.predict(x))

49152/35363 [=========================================] - 0s 1us/step


[[(u'n03729826', u'matchstick', 0.07593459),
  (u'n01930112', u'nematode', 0.048336536),
  (u'n03666591', u'lighter', 0.030769305),
  (u'n03196217', u'digital_clock', 0.02951129),
  (u'n04286575', u'spotlight', 0.023564944)]]